In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import sqlite3

# Test on a chunk of endpoints

In [2]:
endpoint_cols = ['NPI', 'Endpoint Type', 'Endpoint Type Description', 'Endpoint', 'Affiliation', 'Endpoint Description', 'Affiliation Legal Business Name']
endpoint = pd.read_csv('../data/NPPES/endpoint.csv', chunksize = 1000, usecols = endpoint_cols)
endpoint = next(endpoint)

In [3]:
endpoint.columns = [col.lower().replace(' ', '_') for col in endpoint.columns]

In [4]:
endpoint

,npi,endpoint_type,endpoint_type_description,endpoint,affiliation,endpoint_description,affiliation_legal_business_name
0,1154324382,DIRECT,Direct Messaging Address,Richard.Close@SEP.EClinicalDirectPlus.com,N,NaN,NaN
1,1154324382,DIRECT,Direct Messaging Address,rclose13800@MHSDIRECT.NET,N,NaN,NaN
2,1962405175,DIRECT,Direct Messaging Address,fredericstelzer@epgi.allscriptsdirect.net,N,NaN,NaN
3,1699778894,DIRECT,Direct Messaging Address,aawomolo@direct.iuhealth.org,N,NaN,NaN
4,1396748596,DIRECT,Direct Messaging Address,CHRISTIANO.CALDEIRA@hca4.eclinicaldirectplus.com,N,NaN,NaN
...,...,...,...,...,...,...,...
995,1659375590,FHIR,FHIR URL,https://epicsoap.nortonhealthcare.org/,N,NaN,NaN
996,1659375590,DIRECT,Direct Messaging Address,mstaten131498@direct.nortonhealthcare.org,N,NaN,NaN
997,1386648236,DIRECT,Direct Messaging Address,archibald.skemp.p1@direct.arc.nextgenshare.com,N,NaN,NaN
998,1003810953,SOAP,SOAP URL,https://careepicwest.kp.org:14430/Interconnect...,N,Carequality,NaN


# Add relevant endpoints data to db; call table 'affiliations'

In [5]:
db = sqlite3.connect('../data/hopteam.sqlite')    

In [6]:
for chunk in tqdm(pd.read_csv('../data/NPPES/endpoint.csv', chunksize = 100000, usecols = endpoint_cols)):
    chunk.columns = [col.lower().replace(' ', '_') for col in chunk.columns]
    chunk.to_sql('affiliations', db, if_exists = 'append', index = False)

0it [00:00, ?it/s]

In [7]:
db.close()

# Run a query on affiliations
The table seems smaller than I'd expect

In [11]:
query = """

SELECT *
FROM affiliations;

"""

with sqlite3.connect('../data/hopteam.sqlite') as db:
    affiliations = pd.read_sql(query, db)
    
affiliations

,npi,endpoint_type,endpoint_type_description,endpoint,affiliation,endpoint_description,affiliation_legal_business_name
0,1154324382,DIRECT,Direct Messaging Address,Richard.Close@SEP.EClinicalDirectPlus.com,N,None,None
1,1154324382,DIRECT,Direct Messaging Address,rclose13800@MHSDIRECT.NET,N,None,None
2,1962405175,DIRECT,Direct Messaging Address,fredericstelzer@epgi.allscriptsdirect.net,N,None,None
3,1699778894,DIRECT,Direct Messaging Address,aawomolo@direct.iuhealth.org,N,None,None
4,1396748596,DIRECT,Direct Messaging Address,CHRISTIANO.CALDEIRA@hca4.eclinicaldirectplus.com,N,None,None
...,...,...,...,...,...,...,...
397982,1255086849,DIRECT,Direct Messaging Address,manager@mypsych.com,Y,None,"MY PSYCHIATRIST, PLC"
397983,1164177754,DIRECT,Direct Messaging Address,dmeke@suitableforthepeople.net,N,email,None
397984,1679228266,DIRECT,Direct Messaging Address,info@hirschenterprises.net,N,work email,None
397985,1205581899,SOAP,SOAP URL,Therabill,N,SOAP notes,None
